![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace_in_Spark_NLP_DeBertaForZeroShotClassification.ipynb)

## Import DeBertaForZeroShotClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- This feature is only in `Spark NLP 5.2.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DeBerta models trained/fine-tuned for sequence classification via `DebertaV2ForSequenceClassification` or `TFDebertaV2ForSequenceClassification`. We can use these models for zero-shot classification.
  - These models are usually under `Zero-Shot Classification` category and have `deberta` in their labels
  - For zero-shot classification, we will use models trained on the nli data sets. The model should have been trained on the labels `contradiction`, `entailment` and `neutral`.
- Reference: [TFDebertaV2ForSequenceClassification](https://huggingface.co/docs/transformers/model_doc/deberta-v2#transformers.TFDebertaV2ForSequenceClassification)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.39.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 789.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 20.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you ha

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli](https://huggingface.co/aloxatel/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli) model from HuggingFace as an example
- In addition to `TFDebertaV2ForSequenceClassification` we also need to save the `DebertaV2Tokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import TFDebertaV2ForSequenceClassification, DebertaV2Tokenizer
import tensorflow as tf

MODEL_NAME = 'MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli'

tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFDebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME)
except:
  model = TFDebertaV2ForSequenceClassification.from_pretrained(MODEL_NAME, from_pt=True)

# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask"),
          "token_type_ids": tf.TensorSpec((None, None), tf.int32, name="token_type_ids"),
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['deberta.embeddings.position_ids']
- This IS expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDebertaV2ForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2ForSequenceClassification for predictions without further training.
Instructions for updating:
The TensorFlow Distributions library has mov

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 720708
-rw-r--r-- 1 root root      1071 Apr  9 11:57 config.json
drwxr-xr-x 3 root root      4096 Apr  9 11:57 saved_model
-rw-r--r-- 1 root root 737989928 Apr  9 11:57 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 23072
drwxr-xr-x 2 root root     4096 Apr  9 11:57 assets
-rw-r--r-- 1 root root       56 Apr  9 11:57 fingerprint.pb
-rw-r--r-- 1 root root   163441 Apr  9 11:57 keras_metadata.pb
-rw-r--r-- 1 root root 23449198 Apr  9 11:57 saved_model.pb
drwxr-xr-x 2 root root     4096 Apr  9 11:57 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 2420
-rw-r--r-- 1 root root      23 Apr  9 11:54 added_tokens.json
-rw-r--r-- 1 root root     970 Apr  9 11:54 special_tokens_map.json
-rw-r--r-- 1 root root 2464616 Apr  9 11:54 spm.model
-rw-r--r-- 1 root root    1284 Apr  9 11:54 tokenizer_config.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `som.model` from the tokenizer
- All we need is to just copy the `spm.model` to `saved_model/1/assets` which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `spm.model`

In [6]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/spm.model {asset_path}

In [7]:
# get label strings
labels = [model.config.id2label[l] for l, v in model.config.id2label.items()]
print(labels)

with open(asset_path+'/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

['entailment', 'neutral', 'contradiction']


Voila! We have our `spm.model` and `labels.txt` inside assets directory

In [8]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 2412
-rw-r--r-- 1 root root      32 Apr  9 11:57 labels.txt
-rw-r--r-- 1 root root 2464616 Apr  9 11:57 spm.model


## Import and Save DeBertaForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
#restart here
while True:
  pass

In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3


Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `DeBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DeBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [4]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli'

zero_shot_classifier = DeBertaForZeroShotClassification.loadSavedModel(
    '{}/saved_model/1'.format(MODEL_NAME),
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.annotators.classifier.dl.DeBertaForZeroShotClassification.loadSavedModel.
: java.io.FileNotFoundException: file or folder: file:/content/MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli/saved_model/1 not found
	at com.johnsnowlabs.nlp.util.io.ResourceHelper$SourceStream.<init>(ResourceHelper.scala:112)
	at com.johnsnowlabs.nlp.util.io.ResourceHelper$.copyToLocal(ResourceHelper.scala:211)
	at com.johnsnowlabs.ml.util.LoadExternalModel$.modelSanityCheck(LoadExternalModel.scala:137)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.ReadDeBertaForZeroShotDLModel.loadSavedModel(DeBertaForZeroShotClassification.scala:386)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.ReadDeBertaForZeroShotDLModel.loadSavedModel$(DeBertaForZeroShotClassification.scala:384)
	at com.johnsnowlabs.nlp.annotators.classifier.dl.DeBertaForZeroShotClassification$.loadSavedModel(DeBertaForZeroShotClassification.scala:446)
	at com.johnsnowlabs.nlp.an

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [5]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

NameError: name 'zero_shot_classifier' is not defined

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your DebertaV2ForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [6]:
! ls -l {MODEL_NAME}_spark_nlp

total 745560
-rw-r--r-- 1 root root 760972895 Apr  9 12:02 deberta_classification_tensorflow
-rw-r--r-- 1 root root   2464616 Apr  9 12:02 deberta_spp
drwxr-xr-x 4 root root      4096 Apr  9 12:00 fields
drwxr-xr-x 2 root root      4096 Apr  9 12:00 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForZeroShotClassification model 😊

In [7]:
zero_shot_classifier_loaded = DeBertaForZeroShotClassification.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [8]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to harry up and get to the airport. We are going to miss our flight!"]]

# create a DataFrame in PySpark
inputDataset = spark.createDataFrame(text, ["text"])
model = pipeline.fit(inputDataset)
model.transform(inputDataset).select("class.result").show()

+---------+
|   result|
+---------+
|  [music]|
|[weather]|
|  [sport]|
|  [sport]|
|  [music]|
|  [sport]|
|[weather]|
+---------+



That's it! You can now go wild and use hundreds of `DebertaV2ForSequenceClassification` models as zero-shot classifiers from HuggingFace 🤗 in Spark NLP 🚀